# Analysis

This is includes an accuracy check for numbers used in "Here's the county migration that fueled DeSoto's growth", except those pulled directly from the map or graph. See visprep for analysis to reach graph and maps numbers.

In [103]:
import pandas as pd
import glob
pd.options.mode.chained_assignment = None

In [71]:
# load all processed data
for file in glob.glob("../data/processed/*.csv"):
    exec(file.split('/')[-1].replace(".csv", "") + '= pd.read_csv(file)')

> DeSoto has nearly doubled in size since 2000. Whereas, Memphis' Shelby County has grown 1.3% and Mississippi as a whole has grown only 3.2%.

In [115]:
# filter for Mississppi, DeSoto and Shelby
# type: ignore
counties_pop_estimates_2000_2023[
    (counties_pop_estimates_2000_2023["var"] == "POPESTIMATE")
    & (
        (
            (counties_pop_estimates_2000_2023["state_name"] == "Tennessee")
            & (counties_pop_estimates_2000_2023["county_name"] == "Shelby County")
        )
        | (
            (counties_pop_estimates_2000_2023["state_name"] == "Mississippi")
            & (counties_pop_estimates_2000_2023["county_name"] == "DeSoto County")
        )
        | (
            (counties_pop_estimates_2000_2023["state_name"] == "Mississippi")
            & (counties_pop_estimates_2000_2023["county_name"] == "Mississippi")
        )
    )
].pivot_table( # pivot names to columns
    values="value", columns="county_name", index="year"
).pct_change( # get percent change over 23 years
    periods=23
).dropna().apply( # multiple percents by 100 to make more readable
    lambda x: x * 100, axis=1
) # result

county_name,DeSoto County,Mississippi,Shelby County
year,,,
2023,77.744155,3.203762,1.318866


Top 5 and Bottom 5 net migration

In [114]:
# top 5
# type: ignore
acs_desoto_flows_15y.sort_values('movednet', ascending=False).head(5)

,id,county,movedin,movedout,movednet
433,47157,"Shelby County, Tennessee",57545,38355,19190
242,28027,"Coahoma County, Mississippi",2620,40,2580
288,28143,"Tunica County, Mississippi",4090,1545,2545
283,28133,"Sunflower County, Mississippi",2965,720,2245
332,34003,"Bergen County, New Jersey",1735,0,1735


In [113]:
# bottom 5
# type: ignore
acs_desoto_flows_15y.sort_values('movednet', ascending=True).head(5)

,id,county,movedin,movedout,movednet
270,28105,"Oktibbeha County, Mississippi",820,4025,-3205
407,47037,"Davidson County, Tennessee",645,2945,-2300
266,28093,"Marshall County, Mississippi",6335,8355,-2020
253,28059,"Jackson County, Mississippi",195,1960,-1765
406,47035,"Cumberland County, Tennessee",0,1610,-1610


> For the 2006-10 and 2011-2015 ACS, the Census collected the race of migrants, which showed Black movers contributed 61% to the net migration from Shelby to DeSoto. This is slightly disproportionate to the population of Shelby county, which is about 51% Black.

[51% Black pulled from here](https://fred.stlouisfed.org/graph/?g=1pMKk)

In [109]:
# filter for race characteristics
# type: ignore
race_netmoved = acs_desoto_shelby_movednet_10y_chars[
    acs_desoto_shelby_movednet_10y_chars["chars"].isin(
        [
            "Black or African American alone",
            "Asian alone",
            "Other race alone or Two or more races",
            "White alone",
        ]
    )
]

# combine surveys
race_netmoved["total"] = race_netmoved["2010"] + race_netmoved["2015"]

# calculate percent contribution
race_netmoved["percent_contribution"] = race_netmoved["total"].apply(
    lambda x: 100 * x / sum(race_netmoved["total"])
)

race_netmoved  # result

,chars,2010,2015,total,percent_contribution
1,Black or African American alone,953,546,1499,61.510053
4,Asian alone,46,156,202,8.288880
22,Other race alone or Two or more races,162,-188,-26,-1.066886
23,White alone,984,-222,762,31.267952
